In [1]:
import torch
import pandas as pd

from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt

# Data loading and preparation

In [78]:
edges = pd.read_csv("../data/interim/edges.csv")

/tmp/ipykernel_25721/4050122502.py:1: DtypeWarning: Columns (5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  edges = pd.read_csv("../data/interim/edges.csv")


In [79]:
edges

,Unnamed: 0,node1,node2,weight,type,celltype_major,celltype_minor,clone
0,0,AGTAAGCTCCACTAAT-1,AACACCTACTATCGAA-1,0.465604,sc2vis,CAFs,CAFs MSC iCAF.like,diploid
1,1,ATCCATCCAATCGTGA-1,AACACCTACTATCGAA-1,0.707957,sc2vis,Myeloid,Macrophage,diploid
2,2,CGTTATGTCTGTCAAT-1,AACACCTACTATCGAA-1,0.506494,sc2vis,Normal Epithelial,Mature Luminal,diploid
3,3,GACTTACGTTTCTAAG-1,AACACCTACTATCGAA-1,0.417010,sc2vis,CAFs,CAFs MSC iCAF.like,diploid
4,4,GCTATTAGTTAGGCGG-1,AACACCTACTATCGAA-1,0.408076,sc2vis,CAFs,CAFs MSC iCAF.like,diploid
...,...,...,...,...,...,...,...,...
107691,108181,TGTTGGCCTACACGTG-1,AGCCAGCGTTACAGTC-1,0.500000,vis2grid,NaN,NaN,NaN
107692,108182,TGTTGGCCTACACGTG-1,ATACTATCCGAACATA-1,0.500000,vis2grid,NaN,NaN,NaN
107693,108183,TGTTGGCCTACACGTG-1,CGTAAGGATACGGCAC-1,0.500000,vis2grid,NaN,NaN,NaN
107694,108184,TGTTGGCCTACACGTG-1,CTTATTGAGTTCTGTA-1,0.500000,vis2grid,NaN,NaN,NaN


In [80]:
edges[edges["type"] == "sc2vis"][["node1","node2"]].shape[0] == edges[edges["type"] == "sc2vis"][["node1","node2"]].drop_duplicates().shape[0]

True

In [81]:
edges[edges["type"] == "vis2grid"][["node1","node2"]].shape[0] == edges[edges["type"] == "vis2grid"][["node1","node2"]].drop_duplicates().shape[0]

True

In [82]:
edges[["node1","node2"]].shape[0] == edges[["node1","node2"]].drop_duplicates().shape[0]

True

In [83]:
overcl = pd.read_csv("../data/interim/clones_over.csv")

In [84]:
overcl

,cell.names,leiden
0,CGCAAGGGTTAGTGAG-1,16
1,CAAGACTAGCTTGAGG-1,16
2,GCCAATACATTGCCTG-1,10
3,CTTCAATAGATGATTG-1,10
4,GATGCAAGTATTGCGT-1,10
...,...,...
10716,TCTAACCCAGGTATTG-1,2
10717,TCAACTAGTTAGCGCC-1,diploid
10718,CAGGTCAAGACCATTT-1,2
10719,TGGTGCGCACCATCCA-1,2


In [85]:
overcl.columns = ["node1","clone"]

In [86]:
overcl.clone.unique()

array(['16', '10', '0', 'diploid', '12', '8', '13', '14', '1', '7', '15',
       '4', '6', '9', '5', '11', '2', '3'], dtype=object)

In [87]:
cell_type = pd.read_excel("../data/raw/Requested_Cell_Barcode_Type_Matrices.xlsx", sheet_name="scFFPE-Seq")

In [88]:
cell_type.columns = ["node1","cell_type"]

In [89]:
overcl = cell_type.merge(overcl, on = "node1", how = "left")

In [90]:
overcl

,node1,cell_type,clone
0,AAACAAGCAAACGGGA-1,Stromal,NaN
1,AAACAAGCAAATAGGA-1,Macrophages 1,11
2,AAACAAGCAACAAGTT-1,Perivascular-Like,NaN
3,AAACAAGCAACCATTC-1,Myoepi ACTA2+,2
4,AAACAAGCAACTAAAC-1,Myoepi ACTA2+,10
...,...,...,...
27467,TTTGTGAGTGGTTACT-1,DCIS 1,11
27468,TTTGTGAGTGTTCCAG-1,Stromal,NaN
27469,TTTGTGAGTTACTTCT-1,Prolif Invasive Tumor,1
27470,TTTGTGAGTTGTCATA-1,DCIS 2,NaN


In [91]:
edges = edges.drop(columns = ["clone","celltype_major","celltype_minor"])


In [92]:
overcl = edges[edges["type"] == "sc2vis"].merge(overcl,left_on = "node1",right_on ="node1",how = "left")

In [93]:
overcl.clone = overcl.clone.fillna("diploid")

In [94]:
overcl = overcl[["node1","clone","cell_type"]].drop_duplicates()

In [95]:
overcl.cell_type.fillna("unknown", inplace = True)

In [96]:
edges.merge(overcl[["clone","node1", "cell_type"]],left_on = "node1",right_on ="node1",how = "left", validate = 'm:1')

,Unnamed: 0,node1,node2,weight,type,clone,cell_type
0,0,AGTAAGCTCCACTAAT-1,AACACCTACTATCGAA-1,0.465604,sc2vis,diploid,Stromal
1,1,ATCCATCCAATCGTGA-1,AACACCTACTATCGAA-1,0.707957,sc2vis,diploid,Stromal
2,2,CGTTATGTCTGTCAAT-1,AACACCTACTATCGAA-1,0.506494,sc2vis,diploid,DCIS 2
3,3,GACTTACGTTTCTAAG-1,AACACCTACTATCGAA-1,0.417010,sc2vis,diploid,Stromal
4,4,GCTATTAGTTAGGCGG-1,AACACCTACTATCGAA-1,0.408076,sc2vis,diploid,Stromal
...,...,...,...,...,...,...,...
107691,108181,TGTTGGCCTACACGTG-1,AGCCAGCGTTACAGTC-1,0.500000,vis2grid,NaN,NaN
107692,108182,TGTTGGCCTACACGTG-1,ATACTATCCGAACATA-1,0.500000,vis2grid,NaN,NaN
107693,108183,TGTTGGCCTACACGTG-1,CGTAAGGATACGGCAC-1,0.500000,vis2grid,NaN,NaN
107694,108184,TGTTGGCCTACACGTG-1,CTTATTGAGTTCTGTA-1,0.500000,vis2grid,NaN,NaN


In [97]:
edges= edges.merge(overcl[["clone","node1","cell_type"]],left_on = "node1",right_on ="node1",how = "left")

In [98]:
edges[edges["type"] == "sc2vis"][["node1","node2"]].shape[0] == edges[edges["type"] == "sc2vis"][["node1","node2"]].drop_duplicates().shape[0]

True

In [99]:
edges[edges["type"] == "vis2grid"][["node1","node2"]].shape[0] == edges[edges["type"] == "vis2grid"][["node1","node2"]].drop_duplicates().shape[0]

True

In [100]:
edges[["node1","node2"]].shape[0] == edges[["node1","node2"]].drop_duplicates().shape[0]

True

In [101]:
emb_vis = pd.read_csv("../data/interim/embedding_visium_scvi.csv", index_col = 0)
emb_rna = pd.read_csv("../data/interim/embedding_rna2vis_scvi.csv", index_col = 0)

In [102]:
all_nodes_graph = set(edges.node1.to_list() + edges.node2.to_list())
all_nodes_emb = set(emb_vis.index).union(set(emb_rna.index))
all_nodes = list(all_nodes_graph.intersection(all_nodes_emb))
node_encoder = {all_nodes[i]:i for i in range(len(all_nodes))}
emb_vis = emb_vis.loc[emb_vis.index.isin(all_nodes)]
emb_rna = emb_rna.loc[emb_rna.index.isin(all_nodes)]

In [103]:
edges = edges[edges.node1.isin(all_nodes)]

In [104]:
edges = edges[edges.node2.isin(all_nodes)]

In [105]:
edges.node1 = edges.node1.map(node_encoder)

In [106]:
edges.node2 = edges.node2.map(node_encoder)

In [107]:
edges

,Unnamed: 0,node1,node2,weight,type,clone,cell_type
0,0,7867,540,0.465604,sc2vis,diploid,Stromal
1,1,8247,540,0.707957,sc2vis,diploid,Stromal
2,2,3187,540,0.506494,sc2vis,diploid,DCIS 2
3,3,1644,540,0.417010,sc2vis,diploid,Stromal
4,4,4566,540,0.408076,sc2vis,diploid,Stromal
...,...,...,...,...,...,...,...
107691,108181,4469,4650,0.500000,vis2grid,NaN,NaN
107692,108182,4469,8937,0.500000,vis2grid,NaN,NaN
107693,108183,4469,1826,0.500000,vis2grid,NaN,NaN
107694,108184,4469,7934,0.500000,vis2grid,NaN,NaN


In [108]:
emb_vis = emb_vis.rename(index = node_encoder)
emb_rna = emb_rna.rename(index = node_encoder)

In [109]:
edge_index = torch.tensor([edges.node1,
                           edges.node2], dtype=torch.long)

In [110]:
edge_index

tensor([[7867, 8247, 3187,  ..., 4469, 4469, 4469],
        [ 540,  540,  540,  ..., 1826, 7934, 5969]])

In [111]:
edge_weight = torch.tensor(edges.weight.values, dtype=torch.float)

In [112]:
features = pd.concat([emb_vis, emb_rna])

In [113]:
features = features.sort_index()

In [114]:
x = torch.tensor(features.values , dtype=torch.float)

In [115]:
edges[edges.type == "vis2grid"]

,Unnamed: 0,node1,node2,weight,type,clone,cell_type
49430,49920,540,2773,1.0,vis2grid,NaN,NaN
49431,49921,540,6008,1.0,vis2grid,NaN,NaN
49432,49922,540,4397,0.5,vis2grid,NaN,NaN
49433,49923,540,5610,0.5,vis2grid,NaN,NaN
49434,49924,540,53,0.5,vis2grid,NaN,NaN
...,...,...,...,...,...,...,...
107691,108181,4469,4650,0.5,vis2grid,NaN,NaN
107692,108182,4469,8937,0.5,vis2grid,NaN,NaN
107693,108183,4469,1826,0.5,vis2grid,NaN,NaN
107694,108184,4469,7934,0.5,vis2grid,NaN,NaN


In [116]:
edges.clone = edges.clone.fillna("missing")

In [117]:
edges[edges.clone == "missing"]

,Unnamed: 0,node1,node2,weight,type,clone,cell_type
49430,49920,540,2773,1.0,vis2grid,missing,NaN
49431,49921,540,6008,1.0,vis2grid,missing,NaN
49432,49922,540,4397,0.5,vis2grid,missing,NaN
49433,49923,540,5610,0.5,vis2grid,missing,NaN
49434,49924,540,53,0.5,vis2grid,missing,NaN
...,...,...,...,...,...,...,...
107691,108181,4469,4650,0.5,vis2grid,missing,NaN
107692,108182,4469,8937,0.5,vis2grid,missing,NaN
107693,108183,4469,1826,0.5,vis2grid,missing,NaN
107694,108184,4469,7934,0.5,vis2grid,missing,NaN


In [118]:
nodes_atr = edges[["node1","type","cell_type", "clone"]].drop_duplicates()

In [119]:
nodes_atr = nodes_atr.sort_values(by = "node1")

In [120]:
nodes_atr.clone.unique()

array(['diploid', '0', 'missing', '8', '10', '1', '9', '12', '11', '5',
       '7', '13', '4', '2', '6', '16', '14', '15', '3'], dtype=object)

In [121]:
nodes_atr.cell_type = nodes_atr.cell_type.fillna("missing")

In [122]:
nodes_atr

,node1,type,cell_type,clone
17976,0,sc2vis,DCIS 2,diploid
1832,1,sc2vis,Prolif Invasive Tumor,0
105699,2,vis2grid,missing,missing
59678,3,vis2grid,missing,missing
4122,4,sc2vis,Stromal,diploid
...,...,...,...,...
31732,9435,sc2vis,DCIS 2,5
3068,9436,sc2vis,DCIS 1,14
954,9437,sc2vis,DCIS 2,8
96899,9438,vis2grid,missing,missing


In [123]:
from collections import Counter

Counter(nodes_atr.cell_type)

Counter({'DCIS 2': 653,
         'Prolif Invasive Tumor': 289,
         'missing': 4992,
         'Stromal': 1149,
         'DCIS 1': 441,
         'Macrophages 1': 176,
         'Invasive Tumor': 390,
         'T Cell & Tumor Hybrid': 99,
         'Myoepi KRT15+': 171,
         'Myoepi ACTA2+': 376,
         'Stromal & T Cell Hybrid': 91,
         'unknown': 197,
         'CD8+ T Cells': 30,
         'Endothelial': 198,
         'Perivascular-Like': 52,
         'B Cells': 63,
         'IRF7+ DCs': 13,
         'CD4+ T Cells': 41,
         'Mast Cells': 4,
         'Macrophages 2': 13,
         'LAMP3+ DCs': 2})

In [124]:
ct_list = list(nodes_atr.cell_type.unique())
ct_list.remove("missing")

In [125]:
Counter(nodes_atr.clone)

Counter({'diploid': 2309,
         '0': 398,
         'missing': 4992,
         '8': 283,
         '10': 150,
         '1': 193,
         '9': 62,
         '12': 155,
         '11': 95,
         '5': 270,
         '7': 92,
         '13': 71,
         '4': 114,
         '2': 110,
         '6': 78,
         '16': 12,
         '14': 38,
         '15': 11,
         '3': 7})

In [126]:
clone_list = list(nodes_atr.clone.unique())
clone_list.remove("missing")
clone_list.remove("diploid")
clone_dict = {x:int(x) for x in clone_list}
clone_dict["missing"] = -1
clone_dict["diploid"] = len(clone_dict)-1

In [127]:
clone_dict

{'0': 0,
 '8': 8,
 '10': 10,
 '1': 1,
 '9': 9,
 '12': 12,
 '11': 11,
 '5': 5,
 '7': 7,
 '13': 13,
 '4': 4,
 '2': 2,
 '6': 6,
 '16': 16,
 '14': 14,
 '15': 15,
 '3': 3,
 'missing': -1,
 'diploid': 17}

In [128]:
    ct_list = list(nodes_atr.cell_type.unique())
    ct_list.remove("missing")

In [129]:
type_dict = {ct_list[i]:i for i in range(len(ct_list))}

In [130]:
type_dict["missing"] = -1

In [131]:
type_dict

{'DCIS 2': 0,
 'Prolif Invasive Tumor': 1,
 'Stromal': 2,
 'DCIS 1': 3,
 'Macrophages 1': 4,
 'Invasive Tumor': 5,
 'T Cell & Tumor Hybrid': 6,
 'Myoepi KRT15+': 7,
 'Myoepi ACTA2+': 8,
 'Stromal & T Cell Hybrid': 9,
 'unknown': 10,
 'CD8+ T Cells': 11,
 'Endothelial': 12,
 'Perivascular-Like': 13,
 'B Cells': 14,
 'IRF7+ DCs': 15,
 'CD4+ T Cells': 16,
 'Mast Cells': 17,
 'Macrophages 2': 18,
 'LAMP3+ DCs': 19,
 'missing': -1}

In [132]:
nodes_atr.clone = nodes_atr.clone.map(clone_dict)

In [133]:
edges.cell_type = edges.cell_type.fillna("missing")

In [134]:
nodes_atr.cell_type = nodes_atr.cell_type.map(type_dict)

In [135]:
nodes_atr = nodes_atr.set_index("node1")

In [136]:
nodes_atr.head()

,type,cell_type,clone
node1,,,
0,sc2vis,0,17
1,sc2vis,1,0
2,vis2grid,-1,-1
3,vis2grid,-1,-1
4,sc2vis,2,17


In [137]:
features = features.join(nodes_atr)

In [138]:
features.clone = features.clone.fillna(-1)

In [139]:
features.cell_type = features.cell_type.fillna(-1)

In [140]:
features

,0,1,type,cell_type,clone
0,-1.450992,0.556841,sc2vis,0,17
1,0.727731,-0.089595,sc2vis,1,0
2,0.552704,-0.017306,vis2grid,-1,-1
3,0.650890,0.022090,vis2grid,-1,-1
4,-0.123433,0.363432,sc2vis,2,17
...,...,...,...,...,...
9435,-1.727460,0.685548,sc2vis,0,5
9436,1.256450,0.811941,sc2vis,3,14
9437,-1.382968,0.929188,sc2vis,0,8
9438,-0.432557,2.034255,vis2grid,-1,-1


In [141]:
y_clone = torch.tensor(features.clone.values,dtype=torch.long)


In [142]:
y_type = torch.tensor(features.cell_type.values,dtype=torch.long)

In [143]:
y_type.unique()

tensor([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19])

In [144]:
y_clone.unique()

tensor([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17])

In [145]:
data = Data(x=x, edge_index=edge_index, y_clone = y_clone,y_type = y_type, 
edge_type = edges.type.values, edge_attr = edge_weight)

In [146]:
data.validate(raise_on_error=True)


True

In [147]:
full_dict = {}
full_dict["nodes"] = node_encoder
full_dict["clones"] = clone_dict
full_dict["types"] = type_dict


In [148]:
    import pickle   

    torch.save(data, "../data/processed/data.pt")
    with open('../data/processed/full_encoding.pkl', 'wb') as fp:
        pickle.dump(full_dict, fp)

In [149]:
clone_dict

{'0': 0,
 '8': 8,
 '10': 10,
 '1': 1,
 '9': 9,
 '12': 12,
 '11': 11,
 '5': 5,
 '7': 7,
 '13': 13,
 '4': 4,
 '2': 2,
 '6': 6,
 '16': 16,
 '14': 14,
 '15': 15,
 '3': 3,
 'missing': -1,
 'diploid': 17}

In [150]:
type_dict

{'DCIS 2': 0,
 'Prolif Invasive Tumor': 1,
 'Stromal': 2,
 'DCIS 1': 3,
 'Macrophages 1': 4,
 'Invasive Tumor': 5,
 'T Cell & Tumor Hybrid': 6,
 'Myoepi KRT15+': 7,
 'Myoepi ACTA2+': 8,
 'Stromal & T Cell Hybrid': 9,
 'unknown': 10,
 'CD8+ T Cells': 11,
 'Endothelial': 12,
 'Perivascular-Like': 13,
 'B Cells': 14,
 'IRF7+ DCs': 15,
 'CD4+ T Cells': 16,
 'Mast Cells': 17,
 'Macrophages 2': 18,
 'LAMP3+ DCs': 19,
 'missing': -1}

In [151]:
node_encoder

{'ACCAACTGTGCGGCAA-1': 0,
 'GTTTGAGCAAGGTGTC-1': 1,
 'TGGAAGACGTGAGTAG-1': 2,
 'ATACATCACTGTCCGG-1': 3,
 'GAGCCTGCAATAGAGA-1': 4,
 'TCCCTTTAGGCCTTCG-1': 5,
 'GGATCGCTGGCATATT-1': 6,
 'TGTTAGCGTTGGGTTT-1': 7,
 'CTACCAGACCTCTTAG-1': 8,
 'TTCGTTCTCCTGGATC-1': 9,
 'GAAGCGGCTGATCAGT-1': 10,
 'CCTAACTACGAATATG-1': 11,
 'GGCACAATCGCGTTCG-1': 12,
 'CCTGCTCGCCGTCCAG-1': 13,
 'CCGAGTAATACACGAG-1': 14,
 'TGCTTATGAATAAGCA-1': 15,
 'GCACTCAGTTAGTTTG-1': 16,
 'TTCGGTCAGAAGCATC-1': 17,
 'CATAAACCAAAGAACA-1': 18,
 'GAATGTATTGGATCAA-1': 19,
 'TCGTATCCTGATGCAA-1': 20,
 'CATCGGTTCATGGTCC-1': 21,
 'CCGTACGTCCTAATCG-1': 22,
 'CACGATACAATAGCCA-1': 23,
 'GATGCCTAGAGTCTGT-1': 24,
 'CATTTAGGTCCTCCAT-1': 25,
 'AATCCGTGTTGCGGCT-1': 26,
 'ATCCCAATCATGCATT-1': 27,
 'ATATGCAAGCATCTAT-1': 28,
 'CCCACCTTCACTTGAT-1': 29,
 'GTTTATCAGCATTTCG-1': 30,
 'GCTGTCGCGATGGTAG-1': 31,
 'CGTGCTTAGCGAGGGA-1': 32,
 'TCACATAAGCGTTGCC-1': 33,
 'CCGCTCTGGATATCGG-1': 34,
 'TTATTGGGTTTATGGC-1': 35,
 'GTTAAGGGTTCACCTC-1': 36,
 'TCAGGTTGT